Considerando a seguinte formulação:

<center>

$$Minimize f(\overset{\rightarrow}{x}),\overset{\rightarrow}{x} = [ {x_1},{x_2},{...},{x_n} ]$$

</center>

Subject to:

<center>

$$g_i(\overset{\rightarrow}{x}) \leq 0, i = 1,..., q$$

$$h_j(\overset{\rightarrow}{x}) = 0, j = q+1,..., m$$

</center>

Usually equality constraints are transformed into inequalities of the
form

<center>

$$\left| h_j(\overset{\rightarrow}{x}) \right| - \epsilon \leq 0, for j = q + 1,...,m $$

</center>

A solution $\overset{\rightarrow}{x}$ is regarded as *feasible* if
$g_i(\overset{\rightarrow}{x}) \leq 0$, for j = 1,…,q and
$\left| h_j(\overset{\rightarrow}{x}) \right| - \epsilon \leq 0, for$
$j = q + 1,...,m$. In this special session $\epsilon$ is set to 0.0001.

Implementar um algoritmo baseado em Computação Evolutiva (AG, ES, PE ou
PG) ou Evolução Diferencial (ED) ou Particle Swarm Optimization (PSO)
para resolver os seguintes problemas restritos de otimização:


### 1) Problema com 13 variáveis de decisão ($x_1$ até $x_{13}$ ) e 9 restrições de desigualdade.

g01

$Minimize [1]$:

<center>

$$ f(\overset{\rightarrow}{x}) = 5 \sum_{i=1}^4 x_i - 5 \sum_{i=1}^4 x²_i -  \sum_{i=5}^{13} x_i $$

</center>

subject to:

<center>

$$g_1(\overset{\rightarrow}{x})  =  2x_1 + 2x_2 + x_{10} + x_{11} - 10 \leq 0 $$

$$g_2(\overset{\rightarrow}{x}) = 2x_1 + 2x_3 + x_{10} + x_{12} - 10 \leq 0 $$

$$g_3(\overset{\rightarrow}{x}) = 2x_2 + 2x_3 + x_{11} + x_{12} - 10 \leq 0 $$

$$g_4(\overset{\rightarrow}{x}) = -8x_1 + x_{10} \leq 0 $$

$$g_5(\overset{\rightarrow}{x}) = -8x_2 + x_{11} \leq 0 $$

$$g_6(\overset{\rightarrow}{x}) = -8x_3 + x_{12} \leq 0 $$

$$g_7(\overset{\rightarrow}{x}) = -2x_4 - x_5 + x_{10} \leq 0 $$

$$g_8(\overset{\rightarrow}{x}) = -2x_6 - x_7 + x_{11} \leq 0 $$

$$g_9(\overset{\rightarrow}{x}) = -2x_8 - x_9 + x_{12} \leq 0 $$

</center>

com

<center>

$$0 \leq x \leq 1 (i = 1,...,9), 0 \leq  x_i \leq 100  (i = 10, 11, 12) and 0 \leq  x_{13} \leq 1$$

</center>

In [87]:
import random
import math

def evaluate(individual):
    return 5 * sum(individual[1:4]) - 5 * sum([xi**2 for xi in individual[1:4]]) - sum(individual[4:13])

def generate_individual():
    individual = []
    for i in range(13):
        if i < 9:
            individual.append(random.uniform(0, 1))
        elif i < 12:
            individual.append(random.uniform(0, 100))
        else:
            individual.append(random.uniform(0, 1))
    return individual

def generate_population(pop_size):
    return [generate_individual() for _ in range(pop_size)]

def selection(population, tournament_size):
    tournament = random.sample(population, tournament_size)
    return min(tournament, key=evaluate)

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            if i < 9:
                individual[i] = random.uniform(0, 1)
            elif i < 12:
                individual[i] = random.uniform(0, 100)
            else:
                individual[i] = random.uniform(0, 1)
    return individual

def enforce_constraints(individual):
    # Constraint g1
    if 2*individual[0] + 2*individual[1] + individual[9] + individual[10] - 10 > 0:
        individual[9] = max(0, 10 - (2*individual[0] + 2*individual[1] + individual[10]))
        individual[10] = max(0, 10 - (2*individual[0] + 2*individual[2] + individual[10]))
    # Constraint g2
    if 2*individual[0] + 2*individual[2] + individual[9] + individual[11] - 10 > 0:
        individual[9] = max(0, 10 - (2*individual[0] + 2*individual[2] + individual[11]))
        individual[11] = max(0, 10 - (2*individual[0] + 2*individual[2] + individual[11]))
    # Constraint g3
    if 2*individual[1] + 2*individual[2] + individual[10] + individual[11] - 10 > 0:
        individual[10] = max(0, 10 - (2*individual[1] + 2*individual[2] + individual[11]))
        individual[11] = max(0, 10 - (2*individual[1] + 2*individual[2] + individual[11]))
    # Constraint g4
    if -8*individual[0] + individual[9] > 0:
        individual[9] = max(0, 8*individual[0])
    # Constraint g5
    if -8*individual[1] + individual[10] > 0:
        individual[10] = max(0, 8*individual[1])
    # Constraint g6
    if -8*individual[2] + individual[11] > 0:
        individual[11] = max(0, 8*individual[2])
    # Constraint g7
    if -2*individual[3] - individual[4] + individual[9] > 0:
        individual[9] = max(0, -2*individual[3] - individual[4])
    # Constraint g8
    if -2*individual[5] - individual[6] + individual[10] > 0:
        individual[10] = max(0, -2*individual[5] - individual[6])
    # Constraint g9
    if -2*individual[7] - individual[8] + individual[11] > 0:
        individual[11] = max(0, -2*individual[7] - individual[8])
    
    return individual

def genetic_programming(pop_size, num_generations, tournament_size, mutation_rate):
    population = generate_population(pop_size)
    for _ in range(num_generations):
        new_population = []
        for _ in range(pop_size):
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)
            child1 = enforce_constraints(child1)
            child2 = enforce_constraints(child2)
            new_population.extend([child1, child2])
        population = new_population
    best_solution = min(population, key=evaluate)
    return best_solution

best_solution = genetic_programming(pop_size=100, num_generations=100, tournament_size=5, mutation_rate=0.05)
print("Melhor solução encontrada:", best_solution)
print("Valor da função:", evaluate(best_solution))


Melhor solução encontrada: [0.6733365245611177, 0.994763867786948, 0.998613765455275, 0.9994659072394468, 0.9994996474635509, 0.9855952622870898, 0.9956209197953159, 0.9987538385292732, 0.9973181325654763, 2.4812691436844725, 2.2668638593675166, 2.465688712338995, 0.9955891254122187]
Valor da função: -13.150564463761471


In [42]:
import numpy as np

# Função objetivo
def objective_function(x):
    return 5 * np.sum(x[1:4]) - 5 * np.sum(x[1:4]**2) - np.sum(x[4:13])

# Restrições de desigualdade
def inequality_constraints(x):
    constraints = np.zeros(9)
    constraints[0] = 2 * x[0] + 2 * x[1] + x[9] + x[10] - 10
    constraints[1] = 2 * x[0] + 2 * x[2] + x[9] + x[11] - 10
    constraints[2] = 2 * x[1] + 2 * x[2] + x[10] + x[11] - 10
    constraints[3] = -8 * x[0] + x[9]
    constraints[4] = -8 * x[1] + x[10]
    constraints[5] = -8 * x[2] + x[11]
    constraints[6] = -2 * x[3] - x[4] + x[9]
    constraints[7] = -2 * x[5] - x[6] + x[10]
    constraints[8] = -2 * x[7] - x[8] + x[11]
    return constraints

# Restrições de igualdade
def equality_constraints(x):
    return np.array([])  # Não há restrições de igualdade nesse problema

# Função de penalidade estática
def penalty_function(x, penalty_factor):
    constraints = np.maximum(inequality_constraints(x), 0)
    penalty = penalty_factor * np.sum(constraints**2)
    return objective_function(x) + penalty

# Função para gerar uma população inicial
def generate_population(size, num_variables):
    population = np.random.rand(size, num_variables)
    return population

# Função para avaliar a aptidão de cada indivíduo da população
def evaluate_population(population, penalty_factor):
    fitness = np.zeros(population.shape[0])
    for i in range(population.shape[0]):
        fitness[i] = penalty_function(population[i], penalty_factor)
    return fitness

# Função de seleção de pais usando torneio binário
def select_parents(population, fitness, num_parents):
    parents = np.zeros((num_parents, population.shape[1]))
    for i in range(num_parents):
        idx1 = np.random.randint(0, population.shape[0])
        idx2 = np.random.randint(0, population.shape[0])
        if fitness[idx1] < fitness[idx2]:
            parents[i] = population[idx1]
        else:
            parents[i] = population[idx2]
    return parents

# Função de crossover de dois pais para produzir um descendente
def crossover(parent1, parent2):
    crossover_point = np.random.randint(0, parent1.shape[0])
    child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return child

# Função de mutação de um indivíduo
def mutate(individual, mutation_rate):
    for i in range(individual.shape[0]):
        if np.random.rand() < mutation_rate:
            individual[i] = np.random.rand()
    return individual

# Algoritmo genético para resolver o problema
def genetic_algorithm(num_generations, population_size, num_variables, num_parents, mutation_rate, penalty_factor):
    population = generate_population(population_size, num_variables)
    for generation in range(num_generations):
        fitness = evaluate_population(population, penalty_factor)
        parents = select_parents(population, fitness, num_parents)
        offspring = np.zeros((population_size, num_variables))
        for i in range(0, population_size, 2):
            parent1 = parents[i % num_parents]
            parent2 = parents[(i + 1) % num_parents]
            child = crossover(parent1, parent2)
            offspring[i] = mutate(child, mutation_rate)
            if i + 1 < population_size:
                offspring[i + 1] = mutate(child, mutation_rate)
        population = offspring
    best_individual = population[np.argmin(evaluate_population(population, penalty_factor))]
    return best_individual

# Parâmetros do algoritmo genético
num_generations = 100
population_size = 100
num_variables = 13
num_parents = 50
mutation_rate = 0.1
penalty_factor = 1000

# Execução do algoritmo genético
best_solution = genetic_algorithm(num_generations, population_size, num_variables, num_parents, mutation_rate, penalty_factor)

# Imprimir a melhor solução encontrada
print("Melhor solução encontrada:")
print(best_solution)
print("Valor da função objetivo:")
print(objective_function(best_solution))


Melhor solução encontrada:
[0.8705294  0.31187711 0.95941628 0.95602194 0.9047922  0.96633882
 0.96539659 0.7922784  0.77608215 0.80889367 0.88063704 0.87223695
 0.87879487]
Valor da função objetivo:
-6.367498487280373



### 2) Problema com 2 variáveis de decisão ($x_1$ até $x_2$ ) e 5 restrições (2 de desigualdade e 3 de igualdade).

g05

$Minimize [3]$:

<center>

$$f(\overset{\rightarrow}{x}) =  3x_1 + 0.000001x³_1 + 2x_2 + (0.000002/3)x³_2 $$

</center>

subject to:

<center>

$$g_1(\overset{\rightarrow}{x})  =  -x_4 + x_3 - 0.55 \leq 0 $$

$$g_2(\overset{\rightarrow}{x}) = -x_3 + x_4 - 0.55 \leq 0 $$

$$h_3(\overset{\rightarrow}{x}) = 1000sin(-x_3 - 0.25) + 1000sin(-x_4 - 0.25) + 894.8 -x_1   = 0 $$

$$h_4(\overset{\rightarrow}{x}) = 1000sin(x_3 - 0.25) + 1000sin(x_3 - x_4 - 0.25) + 894.8 -x_2   = 0 $$

$$h_5(\overset{\rightarrow}{x}) = 1000sin(x_4 - 0.25) + 1000sin(x_4 - x_3 - 0.25) + 1294.8   = 0 $$

</center>

com

<center>

$$0 \leq x_1 \leq 1200, 0 \leq x_2 \leq 1200, -0.55 \leq x_3 \leq 0.55 and -0.55 \leq x_4 \leq 0.55$$

</center>


In [ ]:
print("")

### Tratamento de Restrições

Terá que ser implementado duas formas de tratamento de restrições, sendo
elas:

-   Penalidade Estática.
-   ɛ-constrained method

### A fazer

Execute o algoritmo genético proposto 30 vezes de modo independente para
cada função objetivo utilizando a Configuração A (Penalidade Estática) e
uma configuração B (tratamento sorteado). E baseado no valor final da
função objetivo retornado em cada uma das 30 execuções faça uma tabela
que mostre: média, valor mínimo, valor máximo e desvio padrão do valor
da função objetivo retornada pelo algoritmo. Mostre também o resultado
graficamente com boxplot. Faça um relatório que explique como os
algoritmos foram implementados (pode ser feito em qualquer linguagem de
programação), quais foram as configurações utilizadas para os parâmetros
da meta-heurística escolhida e como foi feito o tratamento das
restrições em cada problema. Envie também o código fonte. Para a melhor
solução encontrada para cada problema com cada configuração especifique
os valores das variáveis de decisão. Apresente as seguintes tabelas e
discuta os resultados obtidos.